In [1]:
import json
import warnings
import pandas as pd

pd.set_option('future.no_silent_downcasting', True)
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('powerBI/raw_data_with_predictions.csv')

In [3]:
vehicle_price_mapping = {
    'less than 20000': 20000,
    '20000 to 29000': 25000,
    '30000 to 39000': 35000,
    '40000 to 59000': 50000,
    '60000 to 69000': 65000,
    'more than 69000': 75000
}

df['VehiclePrice_average'] = df['VehiclePrice'].map(vehicle_price_mapping)

In [4]:
# df.FraudFound_P.value_counts()

In [5]:
df.to_csv("powerBI/raw_data.csv", index=False)

---

## EBM local scores

In [6]:
with open('powerBI/all_perils_ebm.json', 'r') as f:
    all_perils_ebm = json.load(f)

with open('powerBI/collision_ebm.json', 'r') as f:
    collision_ebm = json.load(f)

In [ ]:
all_perils_ebm['ebm'].keys()

In [ ]:
ap_category_db = pd.DataFrame()
ap_numeric_db = pd.DataFrame()

all_p_intercept = all_perils_ebm['ebm']['intercept']

for feature, term in zip(all_perils_ebm['ebm']['features'], all_perils_ebm['ebm']['terms']):
    if 'categories' in list(feature.keys()):
        print(feature['name'])
        value_df = pd.DataFrame(feature['categories']).T
        value_df.columns=['input_value']
        value_df['feature'] = feature['name']

        value_df['score'] = term['scores'][1:-1]
        value_df['score_std'] = term['standard_deviations'][1:-1]

        ap_category_db = pd.concat([ap_category_db, value_df], axis=0)
    else:
        print(">", feature['name'])
        # numerical feature들을 'binning'하여 score를 부여한 형태
        total_feature_cuts = [feature['min']] + feature['cuts'][0] + [feature['max']]
        tmp = pd.DataFrame(total_feature_cuts, columns=['cut_start'])
        tmp['cut_end'] = tmp.cut_start.shift(-1)
        tmp.dropna(inplace=True)
        tmp['feature'] = feature['name']

        tmp['score'] = term['scores'][1:-1]
        tmp['score_std'] = term['standard_deviations'][1:-1]
        # 따라서 각각의 bin에 따라 score가 분배 되어 있다
        tmp_df_list = list()

        for value in df[feature['name']].unique():
            extracted_tmp = tmp.loc[(tmp['cut_start']<=value) & (tmp['cut_end']>value)]
            extracted_tmp['input_value'] = value
            tmp_df_list.append(extracted_tmp)
        # 가장 마지막 value 값도 추가
        extracted_tmp = tmp.loc[tmp['cut_end']==feature['max']]
        extracted_tmp['input_value'] = feature['max']
        tmp_df_list.append(extracted_tmp)

        value_df = pd.concat(tmp_df_list, axis=0)
        ap_numeric_db = pd.concat([ap_numeric_db, value_df], axis=0)

        # if feature['name']=="DriverRating":
        #     break
        
ap_numeric_db.sort_values(['feature', 'input_value'], inplace=True)
ap_numeric_db.drop(columns=['cut_start', 'cut_end'], inplace=True)
ap_category_db['intercept'] = all_p_intercept[0]
ap_numeric_db['intercept'] = all_p_intercept[0]

In [55]:
ap_numeric_db['policy_type'] = "All Perils"
ap_numeric_db['feature_type'] = "numeric"
ap_category_db['policy_type'] = "All Perils"
ap_category_db['feature_type'] = "category"

all_p_local_scores = pd.concat([ap_numeric_db, ap_category_db], axis=0)

In [ ]:
c_category_db = pd.DataFrame()
c_numeric_db = pd.DataFrame()

collision_intercept = collision_ebm['ebm']['intercept']

for feature, term in zip(collision_ebm['ebm']['features'], collision_ebm['ebm']['terms']):
    if 'categories' in list(feature.keys()):
        print(feature['name'])
        value_df = pd.DataFrame(feature['categories']).T
        value_df.columns=['input_value']
        value_df['feature'] = feature['name']

        value_df['score'] = term['scores'][1:-1]
        value_df['score_std'] = term['standard_deviations'][1:-1]

        c_category_db = pd.concat([c_category_db, value_df], axis=0)
    else:
        print(">", feature['name'])
        # numerical feature들을 'binning'하여 score를 부여한 형태
        total_feature_cuts = [feature['min']] + feature['cuts'][0] + [feature['max']]
        tmp = pd.DataFrame(total_feature_cuts, columns=['cut_start'])
        tmp['cut_end'] = tmp.cut_start.shift(-1)
        tmp.dropna(inplace=True)
        tmp['feature'] = feature['name']

        tmp['score'] = term['scores'][1:-1]
        tmp['score_std'] = term['standard_deviations'][1:-1]
        # 따라서 각각의 bin에 따라 scorer가 분배 되어 있다
        tmp_df_list = list()

        for value in df[feature['name']].unique():
            extracted_tmp = tmp.loc[(tmp['cut_start']<=value) & (tmp['cut_end']>value)]
            extracted_tmp['input_value'] = value
            tmp_df_list.append(extracted_tmp)
        # 가장 마지막 value 값도 추가
        extracted_tmp = tmp.loc[tmp['cut_end']==feature['max']]
        extracted_tmp['input_value'] = feature['max']
        tmp_df_list.append(extracted_tmp)

        value_df = pd.concat(tmp_df_list, axis=0)
        c_numeric_db = pd.concat([c_numeric_db, value_df], axis=0)
        
c_numeric_db.sort_values(['feature', 'input_value'], inplace=True)
c_numeric_db.drop(columns=['cut_start', 'cut_end'], inplace=True)
c_category_db['intercept'] = collision_intercept[0]
c_numeric_db['intercept'] = collision_intercept[0]

In [60]:
c_numeric_db['policy_type'] = "Collision"
c_numeric_db['feature_type'] = "numeric"
c_category_db['policy_type'] = "Collision"
c_category_db['feature_type'] = "category"

collision_local_scores = pd.concat([c_numeric_db, c_category_db], axis=0)

In [62]:
local_scores = pd.concat([collision_local_scores, all_p_local_scores], axis=0)
local_scores.to_csv("powerBI/local_scores.csv", index=False)

---

In [ ]:
local_scores

In [ ]:
products = df[['BasePolicy']].drop_duplicates()
products.to_csv("powerBI/products.csv", index=True)
products

---

In [65]:
time_vars = ["Month", "WeekOfMonth", "DayOfWeek", "DayOfWeekClaimed", 'MonthClaimed', 'WeekOfMonthClaimed', 'Year']
vehicle_vars = ["Make", "VehiclePrice_num", "VehicleCategory", "AgeOfVehicle", "VehiclePrice", "VehiclePrice_average"]
personal_vars = ["Sex", 'MaritalStatus', "Age", 'DriverRating', 'AgeOfPolicyHolder', 'NumberOfCars', 'PastNumberOfClaims', 'AddressChange_Claim']
policy_vars = ["PolicyType", 'Deductible', 'AgentType', "NumberOfSuppliments", 'PolicyNumber', 'RepNumber', 'BasePolicy']
accident_vars = ['Days_Policy_Accident', 'Days_Policy_Claim', 'PoliceReportFiled', 'WitnessPresent', 'AddressChange_Claim_2', 'AccidentArea', 'Fault', 'FraudFound_P', 'predictions']

In [ ]:
time_df

In [66]:
time_df = pd.DataFrame(time_vars, columns=['feature'])
time_df['feature_type'] = 'time'

vehicle_df = pd.DataFrame(vehicle_vars, columns=['feature'])
vehicle_df['feature_type'] = 'vehicle'

personal_df = pd.DataFrame(personal_vars, columns=['feature'])
personal_df['feature_type'] = 'personal'

policy_df = pd.DataFrame(policy_vars, columns=['feature'])
policy_df['feature_type'] = 'policy'

accident_df = pd.DataFrame(accident_vars, columns=['feature'])
accident_df['feature_type'] = 'accident'

features = pd.concat([time_df, vehicle_df, personal_df, policy_df, accident_df], axis=0)

In [ ]:
features

In [ ]:
features.to_csv("powerBI/features.csv", index=False)
features.tail(3)

---